# Harmonise land cover classification

## Description
This notebook uses predictions from previous two years to harmonise and process current years prediction based on a set of customised rules. The implementation is expected to remove unlikely changes between classes within this three-year window time.

## Load Packages

In [ ]:
%matplotlib inline
import numpy as np
import geopandas as gpd
import xarray as xr
import rioxarray
import datacube
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.plotting import rgb
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from skimage.segmentation import expand_labels
from odc.algo import xr_reproject
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,BoundaryNorm
from matplotlib.patches import Patch

## Analysis parameters
* `map_path_current`: File path and name of the post-processed classification map for the current year and entire district produced from previous notebook.
* `map_path_last`: File path and name of the post-processed classification map for last year and entire district produced from previous notebook.
* `map_path_bf_last`: File path and name of the post-processed classification map for the year before last year and entire district produced from previous notebook.
* `dict_map`: A dictionary map of class names corresponding to pixel values.

In [ ]:
map_path_current='Results/Land_cover_prediction_postprocessed.tif'
map_path_last=''
map_path_bf_last=''
dict_map={'Forest':1,'Grassland':5,'Shrubland':7,'Cropland':10,
          'Wetland':11,'Water Body':12,'Urban Settlement':13,'Bare Land':14}

## Load maps

In [ ]:
xr_current=rioxarray.open_rasterio(map_path_current).astype(np.uint8).squeeze()
xr_last=rioxarray.open_rasterio(map_path_current).astype(np.uint8).squeeze()
xr_bf_last=rioxarray.open_rasterio(map_path_current).astype(np.uint8).squeeze()

In [ ]:
ds_geobox=xr_current.geobox
xr_last=xr_reproject(xr_last, ds_geobox, resampling="nearest")
xr_bf_last=xr_reproject(xr_bf_last, ds_geobox, resampling="nearest")

## Apply rules  
The following rules are applied on a per-pixel basis:  

* Reassign the class (if other than Water Body) of the second year to that of previous/following year if the class of the previous and following years is the same;  
* Reassign Urban Settlement to Water Body, if it was Water Body within previous two years;
* Reassign Cropland to Urban Settlement, if it was Urban Settlement within previous two years;
* Reassign Cropland to Bare Land, if it was Bare Land within previous two years;
* Reassign Bare Land to Urban Settlement, if it was Urban Settlement within previous two years;
* Reassign Forest to previous class if it was not Wetland or Shrubland within previous two years;
* Reassign Wetland to previous class if it was not Grassland, Shrubland or Cropland within previous two years;
* Reassign Shrubland to previous class if it was not Wetland, Grassland or Forest within previous two years;
* Reassign Grassland to previous class if it was not Bare Land, Cropland or Wetland within previous two years.

In [ ]:
np_current=xr_current.squeeze().to_numpy()
np_last=xr_last.squeeze().to_numpy()
np_bf_last=xr_bf_last.squeeze().to_numpy()

In [ ]:
np_last=np.where((np_last!=dict_map['Water Body'])&(np_last!=np_bf_last)&(np_current==np_bf_last),np_current,np_last)
np_current=np.where((np_current==dict_map['Urban Settlement'])&(np_last==dict_map['Water Body']),dict_map['Water Body'],np_current)
np_current=np.where((np_current==dict_map['Cropland'])&((np_last==dict_map['Urban Settlement'])|(np_last==dict_map['Bare Land'])),
                    dict_map['Urban Settlement'],np_current)
np_current=np.where((np_current==dict_map['Bare Land'])&(np_last==dict_map['Urban Settlement']),dict_map['Urban Settlement'],np_current)
np_current=np.where((np_current==dict_map['Forest'])&((np_last!=dict_map['Wetland'])&(np_last!=dict_map['Shrubland'])),
                    np_last,np_current)
np_current=np.where((np_current==dict_map['Wetland'])&((np_last!=dict_map['Grassland'])&(np_last!=dict_map['Shrubland'])&(np_last!=dict_map['Cropland'])),
                    np_last,np_current)
np_current=np.where((np_current==dict_map['Shrubland'])&((np_last!=dict_map['Wetland'])&(np_last!=dict_map['Grassland'])&(np_last!=dict_map['Forest'])),
                    np_last,np_current)
np_current=np.where((np_current==dict_map['Grassland'])&((np_last!=dict_map['Bare Land'])&(np_last!=dict_map['Cropland'])&(np_last!=dict_map['Wetland'])),
                    np_last,np_current)

In [ ]:
xr_current_harmonised=xr.DataArray(data=np_current,dims=['y','x'],
                         coords={'y':xr_current.y.to_numpy(), 'x':xr_current.x.to_numpy()})
output_crs=xr_current.rio.crs
xr_current_harmonised.rio.write_crs(output_crs, inplace=True)

## Display results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

colours = {1:'green', 5:'gold', 7:'chocolate',9:'violet',10:'pink',11:'cyan',12:'blue',13:'gray',14:'seashell'}
colours_merged={1:'green', 5:'gold', 7:'chocolate',10:'pink',11:'cyan',12:'blue',13:'gray',14:'seashell'}

patches_list=[Patch(facecolor=colour) for colour in colours.values()]
patches_list_merged=[Patch(facecolor=colour) for colour in colours_merged.values()]

# set color legends and color maps parameters
prediction_values=np.unique(lc_map)
cmap=ListedColormap([colours[k] for k in prediction_values])
norm = BoundaryNorm(list(prediction_values)+[np.max(prediction_values)+1], cmap.N)

prediction_values_merged=np.unique(lc_map_postproc)
cmap_merged=ListedColormap([colours_merged[k] for k in prediction_values_merged])
norm_merged = BoundaryNorm(list(prediction_values_merged)+[np.max(prediction_values_merged)+1], cmap_merged.N)

xr_current.plot.imshow(ax=axes[0], 
               cmap=cmap,
               norm=norm,
               add_labels=True, 
               add_colorbar=False,
               interpolation='none')

xr_current_harmonised.plot.imshow(ax=axes[1], 
               cmap=cmap_merged,
               norm=norm_merged,
               add_labels=True, 
               add_colorbar=False,
               interpolation='none')

# Remove axis on middle and right plot
axes[1].get_yaxis().set_visible(False)
# add colour legends
axes[0].legend(patches_list, list(dict_map.keys()),
    loc='upper center', ncol =4, bbox_to_anchor=(0.5, -0.1))
axes[1].legend(patches_list_merged, list(dict_map.keys()),
    loc='upper center', ncol =4, bbox_to_anchor=(0.5, -0.1))
# Add plot titles
axes[0].set_title('Classification before harmonisation')
axes[1].set_title('Classification after harmonisation')

## Export results

In [ ]:
write_cog(lc_map_postproc, 'Results/Land_cover_prediction_postprocessed_harmonised.tif', overwrite=True)